# Загрузка и первичный препроцессинг данных

In [ ]:
import pandas as pd
import numpy as np

from my_functions import *

import logging
import datetime

import numpy as np
import seaborn as sns

import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle

from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import *
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from pandas.plotting import scatter_matrix

import seaborn as sns
sns.set(font_scale=1.4)

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
# Set notebook style in HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
df_list = []
for fpath in ['../data/agents2019.csv', '../data/agents2020.csv', '../data/agents2021.csv']:
    df_list.append(pd.read_csv(fpath))

In [ ]:
df_list[0].head(5)

In [ ]:
df_list[1].head(5)

In [ ]:
df_list[2].head(5)

## Препроцессинг данных за 2021 год

In [ ]:
df_2021 = df_list[2].copy()

In [ ]:
df_2021 = df_2021.drop(columns=['Unnamed: 0'])
df_2021.index = df_2021['Наименование ДП']
rename_dict = {'Факт.32':'Факт 32',
               'Факт.31':'Факт 31',
            'Факт.23':'Факт 23'}
df_2021 = df_2021.rename(columns=rename_dict)
df_2021 = rename_cols(df_2021)

In [ ]:
df_2021['кол_во_раз_пдз_за_2021_год_шт'] = 0
for idx in df_2021.index:
    for col_name in ['пдз_1_30', 'пдз_31_90', 'пдз_91_365', 'пдз_более_365']:
        df_2021.loc[idx,'кол_во_раз_пдз_за_2021_год_шт'] += df_2021.loc[idx,col_name]

In [ ]:
#Генерация оборачиваемости дебиторской задолженности (год_ОДЗ)
df_2021['2020_одз'] = (2*df_2021['2020_выручка_rub'] /(df_2021['2019_дебиторская_задолженность_rub'] + df_2021['2020_дебиторская_задолженность_rub']))
df_2021['2019_одз'] = (2*df_2021['2019_выручка_rub'] /(df_2021['2018_дебиторская_задолженность_rub'] + df_2021['2019_дебиторская_задолженность_rub']))
df_2021['2018_одз'] = (2*df_2021['2018_выручка_rub'] /(df_2021['2017_дебиторская_задолженность_rub'] + df_2021['2018_дебиторская_задолженность_rub']))
df_2021['2017_одз'] = (2*df_2021['2017_выручка_rub'] /(df_2021['2016_дебиторская_задолженность_rub'] + df_2021['2017_дебиторская_задолженность_rub']))

#Генерация периода погошения дебиторской задолженности (год_пгдз)
df_2021['2020_пгдз'] = 360 / df_2021['2020_одз']
df_2021['2019_пгдз'] = 360 / df_2021['2019_одз']
df_2021['2017_пгдз'] = 360 / df_2021['2018_одз']
df_2021['2018_пгдз'] = 360 / df_2021['2017_одз']

#Генерация признака доля дебиторской задолженности в оборотных активах (год_ддзоа)
df_2021['2020_ддзоа'] = df_2021['2020_дебиторская_задолженность_rub'] / df_2021['2020_оборотные_активы_rub']
df_2021['2019_ддзоа'] = df_2021['2019_дебиторская_задолженность_rub'] / df_2021['2019_оборотные_активы_rub']
df_2021['2018_ддзоа'] = df_2021['2018_дебиторская_задолженность_rub'] / df_2021['2018_оборотные_активы_rub']
df_2021['2017_ддзоа'] = df_2021['2017_дебиторская_задолженность_rub'] / df_2021['2017_оборотные_активы_rub']
df_2021['2016_ддзоа'] = df_2021['2016_дебиторская_задолженность_rub'] / df_2021['2016_оборотные_активы_rub']

#Пятифакторная модель Альтамана для компаний (оценка банкротсвта)
#X1 = Оборотный капитал / Активы = (оборотные_активы_rub - краткосрочные_обязательства_rub) / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)
#X2 = Нераспределенная прибыль(Чистая прибыль) / Активы = прибыль_убыток_от_продажи_rub / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)
#X3 = Операционная прибыль(Прибыль до налогообложения) / Активы = прибыль_убыток_до_налогообложения__rub / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)
#X4 = Балансовая стоимость собственного капитала / Заёмный капитал = (уставный_капитал__rub + прибыль_убыток_от_продажи_rub) / (заёмные_средства_долгосрочные_rub + заёмные_средства_краткосрочные_rub)
#X5 = Выручка / Активы = выручка_rub / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)

#Модель Альтамана для компаний 2016 год
AKTIVS = (df_2021['2016_внеоборотные_активы_rub'] + df_2021['2016_оборотные_активы_rub'] + df_2021['2016_нематериальные_активы_rub'])
X1 = (df_2021['2016_оборотные_активы_rub'] - df_2021['2016_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_2021['2016_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_2021['2016_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_2021['2016_уставный_капитал__rub'] + df_2021['2016_прибыль_убыток_от_продажи_rub']) / (df_2021['2016_заёмные_средства_долгосрочные_rub'] + df_2021['2016_заёмные_средства_краткосрочные_rub'])
X5 = df_2021['2016_выручка_rub'] / AKTIVS
df_2021['2016_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2017 год
AKTIVS = (df_2021['2017_внеоборотные_активы_rub'] + df_2021['2017_оборотные_активы_rub'] + df_2021['2017_нематериальные_активы_rub'])
X1 = (df_2021['2017_оборотные_активы_rub'] - df_2021['2017_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_2021['2017_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_2021['2017_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_2021['2017_уставный_капитал__rub'] + df_2021['2017_прибыль_убыток_от_продажи_rub']) / (df_2021['2017_заёмные_средства_долгосрочные_rub'] + df_2021['2017_заёмные_средства_краткосрочные_rub'])
X5 = df_2021['2017_выручка_rub'] / AKTIVS
df_2021['2017_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2018 год
AKTIVS = (df_2021['2018_внеоборотные_активы_rub'] + df_2021['2018_оборотные_активы_rub'] + df_2021['2018_нематериальные_активы_rub'])
X1 = (df_2021['2018_оборотные_активы_rub'] - df_2021['2018_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_2021['2018_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_2021['2018_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_2021['2018_уставный_капитал__rub'] + df_2021['2018_прибыль_убыток_от_продажи_rub']) / (df_2021['2018_заёмные_средства_долгосрочные_rub'] + df_2021['2018_заёмные_средства_краткосрочные_rub'])
X5 = df_2021['2018_выручка_rub'] / AKTIVS
df_2021['2018_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2019 год
AKTIVS = (df_2021['2019_внеоборотные_активы_rub'] + df_2021['2019_оборотные_активы_rub'] + df_2021['2019_нематериальные_активы_rub'])
X1 = (df_2021['2019_оборотные_активы_rub'] - df_2021['2019_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_2021['2019_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_2021['2019_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_2021['2019_уставный_капитал__rub'] + df_2021['2019_прибыль_убыток_от_продажи_rub']) / (df_2021['2019_заёмные_средства_долгосрочные_rub'] + df_2021['2019_заёмные_средства_краткосрочные_rub'])
X5 = df_2021['2019_выручка_rub'] / AKTIVS
df_2021['2019_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2020 год
AKTIVS = (df_2021['2020_внеоборотные_активы_rub'] + df_2021['2020_оборотные_активы_rub'] + df_2021['2020_нематериальные_активы_rub'])
X1 = (df_2021['2020_оборотные_активы_rub'] - df_2021['2020_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_2021['2020_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_2021['2020_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_2021['2020_уставный_капитал__rub'] + df_2021['2020_прибыль_убыток_от_продажи_rub']) / (df_2021['2020_заёмные_средства_долгосрочные_rub'] + df_2021['2020_заёмные_средства_краткосрочные_rub'])
X5 = df_2021['2020_выручка_rub'] / AKTIVS
df_2021['2016_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_2021['2017_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_2021['2018_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_2021['2019_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_2021['2020_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

df_2021 = df_2021.replace([np.inf, -np.inf], np.nan)

In [ ]:
df_2021.head(5)

## Подготовка объединеных данных за 3 года

In [ ]:
df_total = join_data(df_list)
df_total.head(5)

In [ ]:
#Генерация оборачиваемости дебиторской задолженности (год_ОДЗ)
df_total['2020_одз'] = (2*df_total['2020_выручка_rub'] /(df_total['2019_дебиторская_задолженность_rub'] + df_total['2020_дебиторская_задолженность_rub']))
df_total['2019_одз'] = (2*df_total['2019_выручка_rub'] /(df_total['2018_дебиторская_задолженность_rub'] + df_total['2019_дебиторская_задолженность_rub']))
df_total['2018_одз'] = (2*df_total['2018_выручка_rub'] /(df_total['2017_дебиторская_задолженность_rub'] + df_total['2018_дебиторская_задолженность_rub']))
df_total['2017_одз'] = (2*df_total['2017_выручка_rub'] /(df_total['2016_дебиторская_задолженность_rub'] + df_total['2017_дебиторская_задолженность_rub']))

#Генерация периода погошения дебиторской задолженности (год_пгдз)
df_total['2020_пгдз'] = 360 / df_total['2020_одз']
df_total['2019_пгдз'] = 360 / df_total['2019_одз']
df_total['2017_пгдз'] = 360 / df_total['2018_одз']
df_total['2018_пгдз'] = 360 / df_total['2017_одз']

#Генерация признака доля дебиторской задолженности в оборотных активах (год_ддзоа)
df_total['2020_ддзоа'] = df_total['2020_дебиторская_задолженность_rub'] / df_total['2020_оборотные_активы_rub']
df_total['2019_ддзоа'] = df_total['2019_дебиторская_задолженность_rub'] / df_total['2019_оборотные_активы_rub']
df_total['2018_ддзоа'] = df_total['2018_дебиторская_задолженность_rub'] / df_total['2018_оборотные_активы_rub']
df_total['2017_ддзоа'] = df_total['2017_дебиторская_задолженность_rub'] / df_total['2017_оборотные_активы_rub']
df_total['2016_ддзоа'] = df_total['2016_дебиторская_задолженность_rub'] / df_total['2016_оборотные_активы_rub']

#Пятифакторная модель Альтамана для компаний (оценка банкротсвта)
#X1 = Оборотный капитал / Активы = (оборотные_активы_rub - краткосрочные_обязательства_rub) / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)
#X2 = Нераспределенная прибыль(Чистая прибыль) / Активы = прибыль_убыток_от_продажи_rub / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)
#X3 = Операционная прибыль(Прибыль до налогообложения) / Активы = прибыль_убыток_до_налогообложения__rub / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)
#X4 = Балансовая стоимость собственного капитала / Заёмный капитал = (уставный_капитал__rub + прибыль_убыток_от_продажи_rub) / (заёмные_средства_долгосрочные_rub + заёмные_средства_краткосрочные_rub)
#X5 = Выручка / Активы = выручка_rub / (внеоборотные_активы_rub + оборотные_активы_rub + нематериальные_активы_rub)

#Модель Альтамана для компаний 2016 год
AKTIVS = (df_total['2016_внеоборотные_активы_rub'] + df_total['2016_оборотные_активы_rub'] + df_total['2016_нематериальные_активы_rub'])
X1 = (df_total['2016_оборотные_активы_rub'] - df_total['2016_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_total['2016_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_total['2016_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_total['2016_уставный_капитал__rub'] + df_total['2016_прибыль_убыток_от_продажи_rub']) / (df_total['2016_заёмные_средства_долгосрочные_rub'] + df_total['2016_заёмные_средства_краткосрочные_rub'])
X5 = df_total['2016_выручка_rub'] / AKTIVS
df_total['2016_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2017 год
AKTIVS = (df_total['2017_внеоборотные_активы_rub'] + df_total['2017_оборотные_активы_rub'] + df_total['2017_нематериальные_активы_rub'])
X1 = (df_total['2017_оборотные_активы_rub'] - df_total['2017_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_total['2017_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_total['2017_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_total['2017_уставный_капитал__rub'] + df_total['2017_прибыль_убыток_от_продажи_rub']) / (df_total['2017_заёмные_средства_долгосрочные_rub'] + df_total['2017_заёмные_средства_краткосрочные_rub'])
X5 = df_total['2017_выручка_rub'] / AKTIVS
df_total['2017_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2018 год
AKTIVS = (df_total['2018_внеоборотные_активы_rub'] + df_total['2018_оборотные_активы_rub'] + df_total['2018_нематериальные_активы_rub'])
X1 = (df_total['2018_оборотные_активы_rub'] - df_total['2018_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_total['2018_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_total['2018_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_total['2018_уставный_капитал__rub'] + df_total['2018_прибыль_убыток_от_продажи_rub']) / (df_total['2018_заёмные_средства_долгосрочные_rub'] + df_total['2018_заёмные_средства_краткосрочные_rub'])
X5 = df_total['2018_выручка_rub'] / AKTIVS
df_total['2018_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2019 год
AKTIVS = (df_total['2019_внеоборотные_активы_rub'] + df_total['2019_оборотные_активы_rub'] + df_total['2019_нематериальные_активы_rub'])
X1 = (df_total['2019_оборотные_активы_rub'] - df_total['2019_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_total['2019_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_total['2019_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_total['2019_уставный_капитал__rub'] + df_total['2019_прибыль_убыток_от_продажи_rub']) / (df_total['2019_заёмные_средства_долгосрочные_rub'] + df_total['2019_заёмные_средства_краткосрочные_rub'])
X5 = df_total['2019_выручка_rub'] / AKTIVS
df_total['2019_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

#Модель Альтамана для компаний 2020 год
AKTIVS = (df_total['2020_внеоборотные_активы_rub'] + df_total['2020_оборотные_активы_rub'] + df_total['2020_нематериальные_активы_rub'])
X1 = (df_total['2020_оборотные_активы_rub'] - df_total['2020_краткосрочные_обязательства_rub']) / AKTIVS
X2 = df_total['2020_прибыль_убыток_от_продажи_rub'] / AKTIVS
X3 = df_total['2020_прибыль_убыток_до_налогообложения__rub'] / AKTIVS
X4 = (df_total['2020_уставный_капитал__rub'] + df_total['2020_прибыль_убыток_от_продажи_rub']) / (df_total['2020_заёмные_средства_долгосрочные_rub'] + df_total['2020_заёмные_средства_краткосрочные_rub'])
X5 = df_total['2020_выручка_rub'] / AKTIVS
df_total['2016_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_total['2017_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_total['2018_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_total['2019_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5
df_total['2020_Z'] = 0.717 * X1 + 0.847 * X2 + 3.107 * X3 + 0.420 * X4 + 0.998 * X5

df_total = df_total.replace([np.inf, -np.inf], np.nan)

In [ ]:
df_by_years_total = prepare_data_for_model(df_total)
df_by_years_total.head(5)

In [ ]:
df_by_years_total.describe()

In [ ]:
df_by_years_total = df_by_years_total.reindex(sorted(df_by_years_total.columns), axis=1)
df_by_years_total.head(5)

Выбрасываем колонки "факт"

In [ ]:
cols_to_drop = ['статус']
for col_name in df_by_years_total.columns:
    if 'факт' in col_name:
        cols_to_drop.append(col_name)
        
df_by_years_total = df_by_years_total.drop(columns=cols_to_drop)

In [ ]:
df_by_years_total.describe()

Преобразование данных в тип float, формируем список cur_pdz_cols с колонками, содержащими информацию о ПДЗ для текущего года (для которого делаем прогноз)

In [ ]:
for col_name in df_by_years_total.columns:
    df_by_years_total[col_name] = df_by_years_total[col_name].fillna(df_by_years_total[col_name].median())
#     df_by_years_total[col_name] = df_by_years_total[col_name].fillna(-1)

In [ ]:
df_by_years_total.to_csv('df_by_years_total.csv', index=False)

# Анализ данных

## Анализ данных за 2021 год

In [ ]:
df_2021['fact_pdz'] = df_2021['кол_во_раз_пдз_за_2021_год_шт'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
change_cols = ['внеоборотные_активы_rub', 'выручка_rub', 'заёмные_средства_долгосрочные_rub', 'заёмные_средства_краткосрочные_rub', 'капитал_и_резервы_rub', \
               'кредиторская_задолженность_rub', 'нематериальные_активы_rub', 'оборотные_активы_rub', 'основные_средства__rub', 'прибыль_убыток_до_налогообложения__rub', \
               'прибыль_убыток_от_продажи_rub', 'себестоимость_продаж_rub', 'уставный_капитал__rub']

for col_name in change_cols:
    df_2021['{} 2020 - 2019'.format(col_name)] = df_2021['2020_{}'.format(col_name)] - df_2021['2019_{}'.format(col_name)]

In [ ]:
# Строим боксплот без выбросов
N = len(df_2021.columns)//3 + 1

plt.figure(figsize=(30,8*N))

i=1
for col_name in df_2021.columns:    
    plt.subplot(N,3,i)
    try:
        plt.boxplot([df_2021.query('fact_pdz==0')[col_name].dropna(), df_2021.query('fact_pdz==1')[col_name].dropna()], showfliers=False)
    except:
        pass
    
    if '_Z' in col_name:
        plt.axhline(y=1.23, color='r', linestyle='--', label='bankrupt')
        plt.axhline(y=2.9, color='g', linestyle='--', label='uncertainty')
        plt.legend()
        
    plt.title(col_name)
    i+=1

### Подготовка данных для кластеризации

In [ ]:
cols_to_fillna_0 = []
for col_name in df_2021.columns:
    if 'факт' in col_name:
        cols_to_fillna_0.append(col_name)
        
df_2021[cols_to_fillna_0] = df_2021[cols_to_fillna_0].fillna(0)

### Кластеризация

In [ ]:
# параметры для деления на кластеры
cols = ['макс_пдз_за_2019_год_дней', 'кол_во_просрочек_свыше_5_ти_дней_за_2019_год_шт',
        'кол_во_раз_пдз_за_2019_год_шт', 'макс_пдз_за_2020_год_дней', 'кол_во_просрочек_свыше_5_ти_дней_за_2020_год_шт', 'кол_во_раз_пдз_за_2020_год_шт',
        '2020_нематериальные_активы_rub', '2020_основные_средства__rub',
        '2020_внеоборотные_активы_rub', '2020_дебиторская_задолженность_rub', '2020_оборотные_активы_rub', '2020_уставный_капитал__rub',
        '2020_капитал_и_резервы_rub', '2020_заёмные_средства_долгосрочные_rub', '2020_долгосрочные_обязательства_rub', '2020_заёмные_средства_краткосрочные_rub',
        '2020_кредиторская_задолженность_rub', '2020_краткосрочные_обязательства_rub', '2020_выручка_rub', '2020_себестоимость_продаж_rub',
        '2020_прибыль_убыток_до_налогообложения__rub', '2020_прибыль_убыток_от_продажи_rub',
        'кол_во_раз_пдз_за_2021_год_шт']

n_clust = 3

# диаграмма рассеивания и гистограммы
scatter_matrix(df_2021[cols], alpha=0.5, figsize=(30, 30))
corr = df_2021[cols].corr()

# нормирование данных перед кластеризацией
from sklearn import preprocessing
dataNorm = preprocessing.MinMaxScaler().fit_transform(df_2021[cols].values)
dfNorm = df_2021[cols].copy()
for idx in range(len(dfNorm.index)):
    dfNorm.iloc[idx,:] = dataNorm[idx]

# кластеризация
clusters = AgglClustering(dfNorm, n_clust, plt_dendrogram = True)

In [ ]:
# Строим боксплот без выбросов 
N = len(cols)//3 + 1

plt.figure(figsize=(30,8*N))
df_2021['I'] = dfNorm['I'].copy()

i=1
for col_name in cols:    
    plt.subplot(N,3,i)
    plt.boxplot([df_2021.query('I==0')[col_name], df_2021.query('I==1')[col_name], df_2021.query('I==2')[col_name]], showfliers=False)
    plt.title(col_name)
    i+=1

In [ ]:
# ДИАГРАММЫ РАССЕИВАНИЯ ПО КЛАСТЕРАМ
for i in range(len(cols)-1):
    for j in range(i+1, len(cols)):
        plt.figure(figsize=(10, 8))
        plt.scatter(dataNorm[:,i], dataNorm[:,j], c=clusters.labels_, cmap='viridis')
        plt.colorbar()
        plt.xlabel(cols[i])
        plt.ylabel(cols[j])

In [ ]:
df_2021 = df_2021.drop(columns=['I'])

### Матрица корреляций

In [ ]:
cols = ['макс_пдз_за_2019_год_дней', 'кол_во_просрочек_свыше_5_ти_дней_за_2019_год_шт',
        'кол_во_раз_пдз_за_2019_год_шт', 'макс_пдз_за_2020_год_дней', 'сред_пдз_за_2020_год_дней', 'кол_во_просрочек_свыше_5_ти_дней_за_2020_год_шт', 
        'кол_во_раз_пдз_за_2020_год_шт', '2020_нематериальные_активы_rub', '2020_основные_средства__rub',
        '2020_внеоборотные_активы_rub', '2020_дебиторская_задолженность_rub', '2020_оборотные_активы_rub', '2020_уставный_капитал__rub',
        '2020_капитал_и_резервы_rub', '2020_заёмные_средства_долгосрочные_rub', '2020_долгосрочные_обязательства_rub', '2020_заёмные_средства_краткосрочные_rub',
        '2020_кредиторская_задолженность_rub', '2020_краткосрочные_обязательства_rub', '2020_выручка_rub', '2020_себестоимость_продаж_rub',
        '2020_прибыль_убыток_до_налогообложения__rub', '2020_прибыль_убыток_от_продажи_rub', 'факт_59', 'факт_60',
        'кол_во_раз_пдз_за_2021_год_шт']

plt.figure(figsize=(30, 30))

corr_matrix = df_2021[cols].corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')

plt.title('Correlation matrix')
plt.show()

## Анализ данных за 2019 и 2020 годы

In [ ]:
df = pd.read_csv('df_by_years_total.csv').query('YY != 2021')

In [ ]:
df = df.drop(columns='YY')
for col_name in df.columns:
    if 'итого' in col_name:
        df = df.drop(columns=[col_name])
df.head(5)

In [ ]:
cols = ['кол_во_просрочек_свыше_5_ти_дней_за_год_шт',
 'кол_во_раз_пдз_за_год_шт',
 'макс_пдз_за_год_дней',
 'общая_сумма_пдз_свыше_5_ти_дней_за_год_руб',
 'сред_пдз_за_год_дней']

for col_name in df.columns:
    if 'YY_1' in col_name:
        cols.append(col_name)
cols

In [ ]:
# параметры для различного деления на кластеры
n_clust = 2

# диаграмма рассеивания и гистограммы
scatter_matrix(df[cols], alpha=0.5, figsize=(30, 30))
corr = df[cols].corr()
print(df[cols].corr())

# нормирование данных перед кластеризацией
from sklearn import preprocessing
dataNorm = preprocessing.MinMaxScaler().fit_transform(df[cols].values)
dfNorm = df[cols].copy()
for idx in range(len(dfNorm.index)):
    dfNorm.iloc[idx,:] = dataNorm[idx]

# кластеризация
clusters = AgglClustering(dfNorm, n_clust, plt_dendrogram = True)

In [ ]:
df['I'] = dfNorm['I'].copy()
df.query('I==0').describe()

In [ ]:
df[cols+['I']].groupby('I').mean()

In [ ]:
# Строим боксплот без выбросов
N = len(cols)//3 + 1

plt.figure(figsize=(30,8*N))

i=1
for col_name in cols:    
    plt.subplot(N,3,i)
    plt.boxplot([df.query('I==0')[col_name], df.query('I==1')[col_name]], showfliers=False)
    plt.title(col_name)
    i+=1

In [ ]:
dfNorm.head(5)

In [ ]:
# ДИАГРАММЫ РАССЕИВАНИЯ ПО КЛАСТЕРАМ
for i in range(len(cols)-1):
    for j in range(i+1, len(cols)):
        plt.figure(figsize=(10, 8))
        plt.scatter(dataNorm[:,i], dataNorm[:,j], c=clusters.labels_, cmap='viridis')
        plt.xlabel(cols[i])
        plt.ylabel(cols[j])

In [ ]:
plt.figure(figsize=(30, 30))

corr_matrix = df[cols].corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.3] = 0

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')

plt.title('Correlation matrix')
plt.show()

## Анализ всех данных

In [ ]:
df = pd.read_csv('df_by_years_total.csv')

In [ ]:
list(df.columns)

In [ ]:
df['fact_pdz'] = df['кол_во_раз_пдз_за_год_шт'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
change_cols = ['внеоборотные_активы_rub', 'выручка_rub', 'заёмные_средства_долгосрочные_rub', 'заёмные_средства_краткосрочные_rub', 'капитал_и_резервы_rub', \
               'кредиторская_задолженность_rub', 'нематериальные_активы_rub', 'оборотные_активы_rub', 'основные_средства__rub', 'прибыль_убыток_до_налогообложения__rub', \
               'прибыль_убыток_от_продажи_rub', 'себестоимость_продаж_rub', 'уставный_капитал__rub']

for col_name in change_cols:
    for i in range(1,5):
        df['YY_{}_{}_change'.format(i, col_name)] = df['YY_{}_{}'.format(i, col_name)] - df['YY_{}_{}'.format(i+1, col_name)]

In [ ]:
df.head(5)

In [ ]:
list(df.columns)

In [ ]:
TARGET_NAME = 'fact_pdz'

In [ ]:
# Строим боксплот без выбросов
N = len(df.columns)//3 + 1

plt.figure(figsize=(30,8*N))

i=1
for col_name in df.columns:    
    plt.subplot(N,3,i)
    plt.boxplot([df.query('fact_pdz==0')[col_name].dropna(), df.query('fact_pdz==1')[col_name].dropna()], showfliers=False)
    if '_Z' in col_name:
        plt.axhline(y=1.23, color='r', linestyle='--', label='bankrupt')
        plt.axhline(y=2.9, color='g', linestyle='--', label='uncertainty')
        plt.legend()
        print(col_name, len(df.query('fact_pdz==1 & {} > 1 & {} < 5'.format(col_name, col_name))[col_name].dropna()), df.query('fact_pdz==1')[col_name].min(), df.query('fact_pdz==1')[col_name].max())
#         plt.ylim(2.5,3)
    plt.title(col_name)
    i+=1

**Модель Альтмана позволяет выделить потенциально «плохих» контрагентов и может служить критерием для быстрой проверки контрагента.**

**Небольшая компания с большей вероятностью погасит ДЗ в срок.**

**Если у контрагента ранее была небольшая ПДЗ, скорее всего он и дальше будет возвращать ДЗ в срок.**

**Имеет смысл обратить внимание на факторы 59, 60  (2021 г).**

**Чем меньше изменений в финансовых показателях, тем меньше ПДЗ.**